In [1]:
from datetime import datetime

from flask import Flask, flash, render_template, redirect, request, url_for
from flask_login import (
    LoginManager,
    login_user,
    logout_user,
    login_required,
    current_user,
)

from models import User, Business, Event, Post, Friendship, Attendance, db

In [2]:
app = Flask(__name__)
app.config.from_object('config')  # Load configuration from config.py

with app.app_context():
    db.init_app(app)
    db.create_all()

In [3]:
# Create the User Yoni
with app.test_request_context():
    user = User(name="Yoni")
    db.session.add(user)
    db.session.commit()

In [4]:
# Create our business
with app.test_request_context():
    business = Business(name="Pizzazza", business_type="Restaurant", address="123 Main St")
    db.session.add(business)
    db.session.commit()

In [5]:
# Set Yoni as the manager
with app.test_request_context():
    pizzazza = Business.query.filter_by(name="Pizzazza").first()
    yoni = User.query.filter_by(name="Yoni").first()
    yoni.managed_business = pizzazza
    db.session.commit()

In [6]:
# Create the user Alice, and make her a friend of Yoni
with app.test_request_context():
    alice = User(name="Alice")
    db.session.add(alice)
    yoni = User.query.filter_by(name="Yoni").first()
    friendship = Friendship(originator=yoni, target=alice)
    yoni.friendships.append(friendship)  # This is one way of adding the friendship to the db
    db.session.commit()

In [7]:
with app.test_request_context():
    bob = User(name="Bob")
    db.session.add(bob)
    yoni = User.query.filter_by(name="Yoni").first()
    friendship = Friendship(originator=bob, target=yoni)
    db.session.add(friendship)  # This is another (probably clearer) way of adding the friendship to the db
    db.session.commit()

In [8]:
# Show everything about the user Yoni
with app.test_request_context():
    yoni = User.query.filter_by(name="Yoni").first()
    print(f"{yoni}")
    print(f"{yoni.user_id=}")
    print(f"{yoni.name=}")
    print(f"{yoni.managed_business_id=}")
    print(f"{yoni.managed_business.name=}")
    print(f"{yoni.friendships=}")
    print(f"Yoni's friends are: {','.join([f.target.name for f in yoni.friendships])}")
    print(f"{yoni.target_friendships=}")
    print(f"Yoni is a friend of: {','.join([f.originator.name for f in yoni.target_friendships])}")
    print(f"{yoni.attended_events=}")


<User(name='Yoni')>
yoni.user_id=1
yoni.name='Yoni'
yoni.managed_business_id=1
yoni.managed_business.name='Pizzazza'
yoni.friendships=[<Friendship 1, 2>]
Yoni's friends are: Alice
yoni.target_friendships=[<Friendship 3, 1>]
Yoni is a friend of: Bob
yoni.attended_events=[]


In [9]:
# Create an event at Pizzazza
with app.test_request_context():
    pizzazza = Business.query.filter_by(name="Pizzazza").first()
    event = Event(name="Halloween Party", business=pizzazza, event_type="Party",
                start_time=datetime(2023, 10, 31, 20),
                end_time=datetime(2023, 10, 31, 23))
    db.session.add(event)
    db.session.commit()

In [10]:
# Have Yoni attend the event
with app.test_request_context():
    yoni = User.query.filter_by(name="Yoni").first()
    party = Event.query.filter_by(name="Halloween Party").first()
    # party.attendees.append(yoni) # This is one way of adding the attendance to the db
    yoni.attended_events.append(party)  # This is another (probably clearer) way of adding the attendance to the db
    db.session.commit()


In [11]:
# Show everything about the user Yoni
with app.test_request_context():
    yoni = User.query.filter_by(name="Yoni").first()
    print(f"{yoni.user_id=}")
    print(f"{yoni.attended_events=}")
    for event in yoni.attended_events:
        print(f"Yoni attended the following event:")
        print(f"{event.name=}")
        print(f"{event.business.name=}")
        print(f"{event.business.address=}")
        print(f"{event.start_time=}")
        print(f"{event.end_time=}")


yoni.user_id=1
yoni.attended_events=[<Event 1>]
Yoni attended the following event:
event.name='Halloween Party'
event.business.name='Pizzazza'
event.business.address='123 Main St'
event.start_time=datetime.datetime(2023, 10, 31, 20, 0)
event.end_time=datetime.datetime(2023, 10, 31, 23, 0)


In [12]:
# Show all users
with app.test_request_context():
    users = User.query.all()
    print(f"Our users are: {', '.join(u.name for u in users)}")

Our users are: Yoni, Alice, Bob


In [13]:
# Alternatively, show all users by iterating over the query cursor - note no .all()
with app.test_request_context():
    print("Our users are:")
    for u in User.query:
        print(f"-  {u.name}")

Our users are:
-  Yoni
-  Alice
-  Bob
